<h1><center>STAT/CSE 416 Lab 2: Ridge and LASSO Regression</center></h1>
<center><b>Section:</b> AA/AB</center>
<center><b>Instructor:</b> Datong</center>
<center><b>TA:</b> Long Nguyen</center>
<center><b>Date:</b> October 12, 2023</center>

*Adapted from slides and notebooks from STAT/CSE 416 Spring 2021 TAs.*

## 1. Introduction

This lab will cover:
- Regularization by the $\ell_2$-penalty.
- Regularization by the $\ell_1$-penalty.
- Cross-validation to select hyperparameters.

## 2. Concepts

### 2.1. Why regularization?

**Regularization** generally refers to adjustment to the learning objective or optimization algorithm that will "smoothen" the resulting predictor $\hat{f}$ as an effort to **decrease variance** and **prevent overfitting**. (See [here](https://en.wikipedia.org/wiki/Overfitting) for examples.)

A common framework for regularization involves decreasing the magnitude of the coefficients in the coefficient vector $w$. We use some measure $R(w)$ of the magnitude of the coefficients and add it to the training objective.
$$
\min_{w} \text{RSS}(w) + \lambda R(w).
$$
Note that these terms will often be in competition with one another; do we care more about fitting the data (keeping $\text{RSS}(w)$ small) or having small coefficients (keeping $R(w)$ small)? $\lambda$ controls this trade off.

**Exercise 2.1:**
- Does increasing $\lambda$ increase or decrease the values of the $w$ parameters?
- Please explain why.
- What would we expect to happen if we set $\lambda < 0$?

### 2.2. Ridge vs Lasso

Ridge regression uses the $\ell_2$ penalty, that is
$$
R(w) = ||w||_2^2 = \sum_{j=1}^d |w_j|^2 = |w_1|^2 + ... + |w_d|^2.
$$
For those familiar with vector calculus or linear algebra, this is just the Euclidean length of $w$.
LASSO regression uses the $\ell_1$ penalty, that is
$$
R(w) = ||w||_1 = \sum_{j=1}^d |w_j| = |w_1| + ... + |w_d|.
$$

## 3. Preprocessing

The basic preprocessing steps will be viewing and standardizing the data.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
# Read in weather data and view.
weather = pd.read_csv("weather.csv")
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   STA     20000 non-null  int64  
 1   YR      20000 non-null  int64  
 2   MO      20000 non-null  int64  
 3   DA      20000 non-null  int64  
 4   MAX     20000 non-null  float64
 5   MIN     20000 non-null  float64
dtypes: float64(2), int64(4)
memory usage: 937.6 KB


In [3]:
train_data, test_data = train_test_split(weather, random_state=42)

input_cols = ['STA', 'YR', 'MO', 'DA', 'MIN']

# TODO:
train_X = train_data[input_cols]
train_y = train_data["MAX"]

test_X = test_data[input_cols]
test_y = test_data["MAX"]

In [4]:
train_X.head()

,STA,YR,MO,DA,MIN
5514,50403,44,1,23,72.0
1266,16405,42,7,25,41.0
5864,10002,44,9,5,74.0
15865,62701,43,4,18,70.0
12892,31701,43,5,19,66.0


Check the shape of each set to make sure they make sense!

In [5]:
print("train input data shape:", train_X.shape)
print("train target data shape:", train_y.shape)
print()
print("test input data shape:", test_X.shape)
print("test target data shape:", test_y.shape)

train input data shape: (15000, 5)
train target data shape: (15000,)

test input data shape: (5000, 5)
test target data shape: (5000,)


**Exercise 3.1:** Normalize training and test set input data (`X`) using statistics generated from the training set. To do this, use the [Standard Scaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) from sklearn.

In [6]:
scaler = StandardScaler().fit(train_X, train_y)

# TODO:
train_X_norm = None
test_X_norm = None

In [7]:
# @title
train_X_norm = scaler.transform(train_X)
test_X_norm = scaler.transform(test_X)

## 4. Ridge Regression

We create a [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) linear model with a regularization coefficent of `1.0`.

Note: This coefficent is referred to as "lambda ($\lambda$)" in course material and "alpha $\alpha$" in the `sklearn` docs. They are the same thing!

In [8]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import numpy as np

**Exercise 4.1:** Train the model using the training data and output the training error.

In [9]:
# TODO:
ridge_model = None

In [ ]:
# @title
ridge_model = Ridge(1.0).fit(train_X_norm, train_y)
ridge_model.score(train_X_norm, train_y)

**Exercise 4.2:** Define a function `rmse(mode, X, y)` that calculates the RMSE error for a given mode, input, and target data.

In [11]:
# TODO:
def rmse(model, X, y):
    pass

In [12]:
# @title
def rmse(model, X, y):
    y_pred = model.predict(X)

    rmse_score = np.sqrt(mean_squared_error(y, y_pred))

    return rmse_score

**Exercise 4.3:** Perform 5-fold cross validation with your Ridge model. Output the array of errors (length 5) as well as the mean error. You should use [Cross Validation Score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html?highlight=cross_val_scor) from `sklearn` to do this.

In [ ]:
ridge_CV_scores = cross_val_score(None, None, None, cv=5, scoring=rmse)
ridge_CV_scores

In [ ]:
# @title
# TODO:
ridge_CV_scores = cross_val_score(Ridge(), train_X_norm, train_y, cv=5, scoring=rmse)
ridge_CV_scores

Perform 5-fold cross validation on Ridge models with a range of `alpha` values. For each `alpha`, print the `alpha` value and the corresponding mean CV score.

In [15]:
for reg_coef in [0.1, 1, 10, 100, 1000, 10000]:
    ridge_model = Ridge(alpha=reg_coef)
    ridge_CV_scores = cross_val_score(ridge_model, train_X_norm, train_y, cv=5, scoring=rmse)
    print(reg_coef, ridge_CV_scores.mean(), sep='\t')

0.1	7.10031822766787
1	7.10031833339405
10	7.100326801621388
100	7.101140102573443
1000	7.1707928854639125
10000	9.228692734889147


Take a look at how the weights of Ridge models change as you change the regularization coefficient!

In [16]:
for reg_coef in [1, 100, 10e4, 10e7, 10e12]:
    ridge_model = Ridge(alpha=reg_coef)
    ridge_model.fit(train_X_norm, train_y)
    print(f"Lambda: \t {reg_coef}")
    print(f"Intercept: \t {ridge_model.intercept_}")
    print(f"Coefficients: \t {ridge_model.coef_}")
    print("------------------")

Lambda: 	 1
Intercept: 	 80.9786
Coefficients: 	 [ 0.4859142   0.76510086 -0.41597283  0.0656297  12.93842181]
------------------
Lambda: 	 100
Intercept: 	 80.9786
Coefficients: 	 [ 0.48822098  0.75637265 -0.41015897  0.0641384  12.85274068]
------------------
Lambda: 	 100000.0
Intercept: 	 80.9786
Coefficients: 	 [ 0.15114683  0.03995545  0.00613217 -0.00817072  1.68244246]
------------------
Lambda: 	 100000000.0
Intercept: 	 80.9786
Coefficients: 	 [ 1.87549150e-04  3.61144532e-05  1.87542531e-05 -1.19519522e-05
  1.93545110e-03]
------------------
Lambda: 	 10000000000000.0
Intercept: 	 80.9786
Coefficients: 	 [ 1.87592938e-09  3.61085618e-10  1.87707406e-10 -1.19566423e-10
  1.93574276e-08]
------------------


# 5. Regularization with LASSO

Create a [LASSO](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) linear model with a regularization coefficent of `1.0`.

In [17]:
from sklearn.linear_model import Lasso

In [18]:
lasso_model = Lasso().fit(train_X_norm, train_y)
rmse(lasso_model, train_X_norm, train_y)

7.248086829452871

Perform 5-fold cross validation with your LASSO model. Output the array of errors (length 5) as well as the mean error.

In [19]:
lasso_CV_scores = cross_val_score(Lasso(), train_X_norm, train_y, cv=5, scoring=rmse)

Perform 5-fold cross validation with your Ridge model. Output the array of errors (length 5) as well as the mean error. You should use [Cross Validation Score](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html?highlight=cross_val_scor) from `sklearn` to do this.

In [20]:
for reg_coef in [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 0.1, 1, 10, 100, 1000]:
    lasso_model = Lasso(alpha=reg_coef)
    lasso_CV_scores = cross_val_score(lasso_model, train_X_norm, train_y, cv=5, scoring=rmse)
    print(reg_coef, lasso_CV_scores.mean(), sep='\t')

1e-06	7.1003182250286
1e-05	7.10031823213338
0.0001	7.100318305065085
0.001	7.100319320447997
0.01	7.100359168439399
0.1	7.102948831090839
1	7.249428309445422
10	12.310905347585491
100	14.766690347305012
1000	14.766690347305012


Take a look at how the weights of LASSO models change as you change the regularization coefficient! For reasons we might not delve into in this course, LASSO tends to find **sparse** solutions, i.e. solutions that have many zeros in the coefficient vector.

In [21]:
for reg_coef in [0.001, 0.1, 1, 10, 100]:
    lasso_model = Lasso(alpha=reg_coef)
    lasso_model.fit(train_X_norm, train_y)
    print(lasso_model.intercept_, lasso_model.coef_)

80.9786 [ 0.48503587  0.76436142 -0.41508768  0.06461611 12.93823835]
80.9786 [ 0.40209085  0.68253688 -0.32186531  0.         12.83413332]
80.9786 [ 0.          0.         -0.          0.         11.90495174]
80.9786 [ 0.          0.         -0.         -0.          2.90495174]
80.9786 [ 0.  0.  0. -0.  0.]


**Exercise 5.1:** Using the regularization coefficient that leads to the best validation error, compute test scores for a Ridge and LASSO model.

In [ ]:
# TODO:
print("Ridge", rmse(Ridge(alpha=None).fit(train_X_norm, train_y), test_X_norm, test_y))
print("LASSO", rmse(Lasso(alpha=None).fit(train_X_norm, train_y), test_X_norm, test_y))

In [ ]:
# @title
# We can see that alpha = 0.1 is best for Ridge and alpha 0.01 is best for Lasso, so we can plug in and find out rmse for testing data of both model.

print("Ridge", rmse(Ridge(alpha=0.1).fit(train_X_norm, train_y), test_X_norm, test_y))
print("LASSO", rmse(Lasso(alpha=0.01).fit(train_X_norm, train_y), test_X_norm, test_y))